# <center>Efficient multigrid methods for high-order nonlinear solid mechanics on emerging architectures</center>

<br>
<br>
<br>

## <center>Rezgar Shakeri<sup>1</sup>, 
<br>
<center>Valeria Barra<sup>2</sup>, Natalie Beams<sup>3</sup>, Jed Brown<sup>4</sup>, Yohann Dudouit<sup>5</sup>, Karen Stengel<sup>4</sup>, Jeremy L. Thompson<sup>4</sup></center>  

<br>
  
<center><sub><sup><sup>1</sup> Department of Civil Engineering, CU Boulder</sup></sub></center>  
<center><sub><sup><sup>2</sup> Department of Environmental Science and Engineering, California Institute of Technology</sup></sub></center>
<center><sub><sup><sup>3</sup> Innovative Computing Laboratory, University of Tennessee</sup></sub></center>  
<center><sub><sup><sup>4</sup> Department of Computer Science, CU Boulder</sup></sub></center>  
<center><sub><sup><sup>5</sup> Lawrence Livermore National Laboratory</sup></sub></center>  

<br>
<br>
<br>

<center>Copper-Iterative, 07 Apr 2022</center>

In [12]:
from IPython.display import SVG, Video, HTML, IFrame

# Outline

### Why Matrix Free ?

### Model Equations

* Neo-Hookean at hyperelasticity at finite strain
* Initial vs Current Configuration

### Matrix Free FE

* Math
* libCEED

### P-Multigrid

### Results


<img src="schwarz-compression.gif" width="80%" height="60%" align="bottom">

# Why Matrix Free ?
<img src="flop-per-byte-dp-2022.svg" width="50%" height="40%" align="bottom">

* Modern hardware does 10 flops per byte.

* Matrix-free methods store and move less data, compute faster.

* Assembled matrices need at least 4 bytes transferred per flop.

# Model Equations

* Strong form
\begin{equation}
-\nabla_X \cdot \boldsymbol{P} - \rho_0 \boldsymbol{g} = 0
\end{equation}
where $\boldsymbol{P}=\boldsymbol{FS}$ and
$$
\color{blue}{
\boldsymbol{S}=\lambda \, \log J \, \boldsymbol{C}^{-1} + \mu \left( \boldsymbol{I} - \boldsymbol{C}^{-1} \right).}
$$

* Variational form ($\color{red}{\text{Initial}}$ configuration)
\begin{equation}
\int_{\Omega_0}{\nabla_X \boldsymbol{v} \colon \boldsymbol{FS}} \, dV 
 - \int_{\Omega_0}{\boldsymbol{v} \cdot \rho_0 \boldsymbol{g}} \, dV
 = 0, \quad \forall \boldsymbol v \in \mathcal V,
\end{equation}

* Variational form ($\color{red}{\text{Current}}$ configuration)
\begin{equation}
\int_{\Omega_0}{\nabla_x \boldsymbol{v} \colon \boldsymbol{\tau}} \, dV 
 - \int_{\Omega_0}{\boldsymbol{v} \cdot \rho_0 \boldsymbol{g}} \, dV
 = 0, \quad \forall \boldsymbol v \in \mathcal V,
\end{equation}
where
$$
\color{teal}{
\boldsymbol{\tau}=\lambda \, \log J \, \boldsymbol{I} + \, \mu \left( \boldsymbol{b}-\boldsymbol{I} \right).}
$$

<img src="initial-vs-current.png" width="100%" height="100%" align="bottom">


### Matrix-free Finite Element Formulation

\begin{gather*}
    v^T F(u) \sim \int_\Omega v \cdot \color{olive}{f_0(u, \nabla u)} + \nabla v \!:\! \color{olive}{f_1(u, \nabla u)} \quad
    v^T J du \sim \int_\Omega \begin{bmatrix} v \\ \nabla v \end{bmatrix}^T \color{teal}{\begin{bmatrix} f_{0,0} & f_{0,1} \\ f_{1,0} & f_{1,1} \end{bmatrix}}
    \begin{bmatrix} du \\ \nabla du \end{bmatrix} \\
\end{gather*}

### [libCEED](https://libceed.readthedocs.io): Efficient Extensible Discretization

<img align="center" src="libCEEDAPI.png" width="50%" height="30%"/>

* <font color='red'>$\mathcal P$</font>: Process decomposition,

* $\mathcal E$</font> : Element restriction,

* <font color='blue'>$B$</font> : Basis (DoFs-to-Qpts) evaluator, 

* <font color='green'>$D$</font> : Operator at quadrature point 


## [libCEED](https://libceed.readthedocs.io): fast algebra for finite elements

* Backend plugins with run-time selection
  * debug and optimized built-in backends
  * libxsmm
  * CUDA (with and without NVRTC)
  * MAGMA to CUDA and HIP
  * OCCA to OpenMP, OpenCL, CUDA, and HIP
* Single source vanilla C for QFunctions
  * Easy to debug and understand locally
* 2-clause BSD, Python interface
* Available via MFEM, PETSc, and Nek5000


Thanks to many developers, including Jeremy Thompson, Yohann Dudouit, Natalie Beams, Valeria Barra, Leila Ghaffari, Tzanio Kolev, Veselin Dobrev, David Medina

<img align="center" src="libCEEDBackends.png" width="100%" height="100%"/>

# P-Multigrid

* h-multigrid vs p-multigrid

<img align="center" src="p-h-multigrid.png" width="100%" height="100%"/>


* P-Multigrid V-Cycle
* 2nd order Chebyshev/Jacobi iteration in the range $[0.1\lambda_{max}, 1.1\lambda_max]$
* $\lambda_{max}$ estimate eigenvalue of $\left(\text{diag}\boldsymbol{A}_f \right)^{-1}\boldsymbol{A}_f$
<img align="center" src="p-multigrid-cycle.png" width="100%" height="100%"/>


# Assembled vs Matrix Free

<img align="center" src="assemble-vs-matrixfree.png" width="80%" height="60%"/>


<img align="center" src="preconditioner.png" width="80%" height="60%"/>


# Flame Graph

In [23]:
IFrame("schwarz-q2-cuda-flame.svg", width="1600", height="400")

# high-order FE in engineering tolarance

* mesh A

<img align="center" src="meshA.png" width="50%" height="40%"/>

* mesh B

<img align="center" src="meshB.png" width="50%" height="40%"/>


<img align="center" src="meshA_deformed.png" width="100%" height="100%"/>


<img align="center" src="accuracy_study_annotated.png" width="1400" height="400"/>


# Future works

## Buckling problem

<img src="schwarz-twist-vonMises.gif" width="400%" height="300%" align="bottom">

* implement arc-length method

# ratel

<img align="left" src="ratel-gitlab.png" width="100%"/>

[https://gitlab.com/micromorph/ratel](https://gitlab.com/micromorph/ratel)

<img align="center" src="baby-ratel.jpeg" width="80%"/>